In [135]:
from dimod import ConstrainedQuadraticModel, Real, Integer, Binary, quicksum

def mps2cmq(filename):
    ### read the mps file ###
    mpsFile = open(filename, "r")
    mpsLine = mpsFile.readlines()
    mpsLine = [s.replace('\n', '') for s in mpsLine]

    readRow, readCol, readInt, readRHS, readBnd, readObj = False, False, False, False, False, False
    equations, bi_variables, int_variables, real_variables, objEq = {}, set(), set(), set(), None
    LHSeq, RHSeq, upBound, lowBound = {}, {}, {}, {}

    for i in range(len(mpsLine)):
        string = mpsLine[i]  
        if  string == 'ROWS':
            readRow = True
        elif string == 'COLUMNS':
            readRow = False
            readCol = True
            LHSeq = {k: [] for k in equations.keys()}
            RHSeq = {k: [] for k in equations.keys()}
        elif string == 'RHS':
            readCol = False
            readRHS = True
        elif string == 'BOUNDS':
            readRHS = False
            readBnd = True

        if readRow and string != 'ROWS':
            relation, eq = string.split()
            equations[eq] = relation
            if not readObj and relation == 'N': ObjEq = eq
        
        if readCol and string != 'COLUMNS':
            strList = string.split()
            if strList[1] == "\'MARKER\'" and strList[2] == "\'INTORG\'":
                readInt = True
                continue
            elif strList[1] == "\'MARKER\'" and strList[2] == "\'INTEND\'":
                readInt = False
                continue

            var = strList[0]
            if readInt:
                int_variables.add(var)
            else:
                real_variables.add(var)
            for i in range(0, len(strList)//2):
                LHSeq[strList[i*2+1]].append((var, strList[i*2+2]))
        
        if readRHS and string != 'RHS':
            strList = string.split()
            for i in range(0, len(strList)//2):
                RHSeq[strList[i*2+1]] = strList[i*2+2]
        
        if readBnd and string != 'BOUNDS':
            strList = string.split()
            if strList[0] == 'UP':
                upBound[strList[2]] = strList[3]
            elif strList[0] == 'LO':
                lowBound[strList[2]] = strList[3]

    mpsFile.close()

    ### convert into cqm ###
    cqm = ConstrainedQuadraticModel()

    # variables
    for s in upBound:
        if upBound[s] == '1' and s in int_variables:
            bi_variables.add(s)
            int_variables.remove(s)

    x = {v: Real(v) for v in real_variables}
    x_int = {v: Integer(v) for v in int_variables}
    x_bi = {v: Binary(v) for v in bi_variables}
    x.update(x_int)
    x.update(x_bi)

    # the objective function and constraint equations
    for eq in equations:
        currLHS = quicksum(x[v]*float(coef) for v, coef in LHSeq[eq])

        if eq == ObjEq:
            cqm.set_objective(currLHS)
        if RHSeq[eq]:
            if equations[eq] == 'E':
                cqm.add_constraint(currLHS == float(RHSeq[eq]), label=eq)
            elif equations[eq] == 'G':
                cqm.add_constraint(currLHS >= float(RHSeq[eq]), label=eq)
            elif equations[eq] == 'L':
                cqm.add_constraint(currLHS <= float(RHSeq[eq]), label=eq)

    # constraints on the variables
    for k in upBound.keys():
         if k not in bi_variables: cqm.set_upper_bound(k, float(upBound[k]))
    for k in lowBound.keys():
        cqm.set_lower_bound(k, float(lowBound[k]))
    
    return cqm

In [136]:
from dwave.system import LeapHybridCQMSampler

# read mps file
# cqm1 = mps2cmq('testprob.mps')
cqm1 = mps2cmq('markshare_4_0.mps')
# cqm1 = mps2cmq('50v-10.mps')

In [137]:
# run the solver
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm1, label='markshare_4_0')

In [138]:
# analyze the results
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)
feasible_sampleset

SampleSet(rec.array([([  6.,  12.,   2.,   4.,   0.,   1.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   1.,   0.,   1.,   1.,   1.,   1.,   0.,   1.,   1.,   0.,   0.,   0.,   1.,   1.,   0.,   1.,   0.],  24., 1, [ True,  True,  True,  True],  True),
           ([  6.,   3.,  14.,  10.,   1.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   1.,   1.,   1.,   0.,   1.,   0.,   1.,   1.,   1.,   0.,   1.,   0.,   0.,   1.,   0.,   1.,   0.,   1.],  33., 1, [ True,  True,  True,  True],  True),
           ([ 14.,   7.,   8.,   4.,   0.,   0.,   1.,   1.,   0.,   1.,   1.,   1.,   0.,   1.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   1.,   1.,   1.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   0.,   0.],  33., 1, [ True,  True,  True,  True],  True),
           ([  7.,  16.,  10.,   0.,   1.,   1.,   0.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,   0.,   1.,   0.,   0.,   1.,   1.,   0.,   0.,   1.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   

In [139]:
feasible_sampleset.first.energy

24.0

In [140]:
for v in cqm1.variables:
    print(v + ": " + str(cqm1.vartype(v)))

s1: Vartype.REAL
x1: Vartype.BINARY
x2: Vartype.BINARY
x3: Vartype.BINARY
x4: Vartype.BINARY
x5: Vartype.BINARY
x6: Vartype.BINARY
x7: Vartype.BINARY
x8: Vartype.BINARY
x9: Vartype.BINARY
x10: Vartype.BINARY
x11: Vartype.BINARY
x12: Vartype.BINARY
x13: Vartype.BINARY
x14: Vartype.BINARY
x15: Vartype.BINARY
x16: Vartype.BINARY
x17: Vartype.BINARY
x18: Vartype.BINARY
x19: Vartype.BINARY
x20: Vartype.BINARY
x21: Vartype.BINARY
x22: Vartype.BINARY
x23: Vartype.BINARY
x24: Vartype.BINARY
x25: Vartype.BINARY
x26: Vartype.BINARY
x27: Vartype.BINARY
x28: Vartype.BINARY
x29: Vartype.BINARY
x30: Vartype.BINARY
s2: Vartype.REAL
s3: Vartype.REAL
s4: Vartype.REAL
